In [14]:
# HMM, 有监督学习算法
# 预测：维比特算法，动态规划
import numpy as np
import time

def trainParameter(filepath):
    # 定义一个查询字典，B代表词语的开头，M代表中间，E代表结尾，S代表单个字的词
    statuDict = {'B':0, 'M':1, 'E':2, 'S':3}
    # PI参数 初试状态概率向量
    PI = np.zeros(4)
    # A 状态转移概率矩阵
    A = np.zeros((4,4))
    # B 观测概率矩阵 每个状态可能出现的字，这里用一个65536的空间来保证对于所有的汉字都能
    #找到对应的位置来存储
    B = np.zeros((4, 65536))
    # 加载文件
    f = open(filepath, encoding='utf-8')
    
    for line in f.readlines():
        #假设一行的内容是：line = ‘深圳  有  个  打工者  阅览室\n’
        curLine = line.strip().split()
        # curLine = ['深圳','有','个', '打工者', '阅览室']
        # 将curline的内容转换成BMES
        # 保存一个序列的标签
        wordLabel = [] 
        for i in range(len(curLine)):
            if len(curLine[i]) == 1:
                label = 'S'
            else:
                label = 'B' + 'M' * (len(curLine[i]) - 2) + 'E'
            # 更新PI
            if i == 0: PI[statuDict[label[0]]] += 1
            # 更新 B ord(str)返回字符对应的十进制整数
            for k in range(len(label)):
                B[statuDict[label[k]]][ord(curLine[i][k])] += 1
            
            wordLabel.extend(label)
            # extend()和label的区别，extend('BMES') 得到 ['B','M', 'E', 'S']
            #而append('BMES') 得到['BMES'] 这里‘BMES’等同于list，append是将整个list添加到后面
        # 等一个序列['B','E','S','M','S','E'] 结束后再更新转移概率矩阵 A
        for i in range(1, len(wordLabel)):
            A[statuDict[wordLabel[i-1]]][statuDict[wordLabel[i]]] += 1
            
    # 上述三个参数的数值还是次数，转换为频率
    # 转换PI
    PI_sum = np.sum(PI)
    for i in range(len(PI)):
        #如果某元素没有出现过，该位置为0，在后续的计算中这是不被允许的
        #比如说某个汉字在训练集中没有出现过，那在后续不同概率相乘中只要有
        #一项为0，其他都是0了，此外整条链很长的情况下，太多0-1的概率相乘
        #不管怎样最后的结果都会很小，很容易下溢出
        #所以在概率上我们习惯将其转换为log对数形式，这在书上是没有讲的
        #x大的时候，log也大，x小的时候，log也相应小，我们最后比较的是不同
        #概率的大小，所以使用log没有问题

        #那么当单向概率为0的时候，log没有定义，因此需要单独判断
        #如果该项为0，则手动赋予一个极小值
        if PI[i] == 0.: PI[i] = -3.14e+100
        else:  PI[i] = np.log(PI[i] / PI_sum)
    # 转换A
    A_sum = np.sum(A, axis=1) # A_sum array([5,9,10,2])
    for i in range(A.shape[0]):
        for k in range(A.shape[1]):
            if A[i][k] == 0:  A[i][k] = -3.14e+100
            else: A[i][k] = np.log(A[i][k] / A_sum[i])
    # 转换B
    B_sum = np.sum(B, axis=1)
    for i in range(B.shape[0]):
        for k in range(B.shape[1]):
            if B[i][k] == 0: B[i][k] = -3.14e+100
            else: B[i][k] = np.log(B[i][k] / B_sum[i])
    
    return PI, A, B

# 加载文章
def loadArtical(filepath):
    '''
    加载文章
    返回文章内容：['今天早上天气很好！'， '深圳有个打工者阅览室。']
    '''
    artical = []
    f = open(filepath, encoding='utf-8')
    for line in f.readlines():
        line = line.strip()
        artical.append(line)
    return artical

#文章分词函数
# 算法，维特比算法
def tokenize(artical, PI, A, B):
    '''
    输入：文章内容：['今天早上天气很好！'， '深圳有个打工者阅览室。'], 参数：PI(array)， A(array)， B(array)
    输出：分词结果：['今天｜早上｜天气｜很｜好｜！'， '深圳｜有｜个｜打工者｜阅览室｜。']
    '''
    tokenization = []
    for line in artical:
        delta = [[0.] * A.shape[0] for i in range(len(line))]
        # 初始化delta
        for k in range(A.shape[0]):
            # 因为概率都取了log，所以这里是加而不是书上的相乘
            delta[0][k] = PI[k] + B[k][ord(line[0])]
        # 初始化Psi, 维度 T*A.shape[0]
        Psi = [[0.] * A.shape[0] for i in range(len(line))]
        # 进行递推
        for t in range(1, len(line)):
            for i in range(A.shape[0]):
                # 临时存放四个状态的概率值
                tempDelta = [0.] * A.shape[0] 
                for j in range(A.shape[0]):
                    tempDelta[j] = delta[t-1][j] + A[j][i]
                # 记录最大的值
                maxDelta = max(tempDelta)
                # 记录t-1最大值对应的索引
                maxDeltaIndex = tempDelta.index(maxDelta)
                delta[t][i] = maxDelta + B[i][ord(line[t])]
                Psi[t][i] = maxDeltaIndex
        # 创建一个序列表用来生成状态链
        sequence = []
        sequence_prob = max(delta[len(line)-1])
        Index_opt = delta[len(line)-1].index(sequence_prob)
        sequence.append(Index_opt)
        for t in range(len(line)-1, 0, -1):
            Index_opt = Psi[t][Index_opt]
            sequence.append(Index_opt)
        sequence.reverse()
        CurLine = ''
        for i in range(len(line)):
            CurLine += line[i]
            # 遇到E：2或者S：3 添加｜，但是改行的结尾出不需要加
            if (sequence[i] == 2 or sequence[i] == 3) and i != len(line) - 1:
                CurLine += '|'
        tokenization.append(CurLine)
    
    return tokenization

In [15]:
PI, A, B = trainParameter('HMMTrainSet.txt')

In [16]:
A

array([[-3.14000000e+100, -1.91884919e+000, -1.58732900e-001,
        -3.14000000e+100],
       [-3.14000000e+100, -1.06226695e+000, -4.24145455e-001,
        -3.14000000e+100],
       [-7.22823902e-001, -3.14000000e+100, -3.14000000e+100,
        -6.64325843e-001],
       [-5.60300594e-001, -3.14000000e+100, -3.14000000e+100,
        -8.46385515e-001]])

In [17]:
artical = loadArtical('testArtical.txt')

In [18]:
tokenization = tokenize(artical, PI, A, B)

In [19]:
for i in tokenization:
    print(i)

深圳|有个|打|工者|阅览室
去年|１２月|，|我|在|广东|深圳|市出|差|，|听|说|南山区|工商|分局|为|打|工者|建了|个|免费|图书|阅览室|，|这件|新|鲜事|引起|了|我|的|兴趣|。
１２月|１８日|下午|，|我来|到|了|这个|阅览室|。|阅览室|位|于|桂庙|，|临南油|大道|，|是|一间|轻|体房|，|面积|约|有４０平|方米|，|内|部装|修得|整洁|干净|，|四|周|的|书架|上|摆满|了|书|，|并|按|政治|、|哲学|、|法律|法规|、|文化|教育|、|经济|、|科技|、|艺术|、|中国|文学|、|外国|文学|等|分类|，|屋|中央|有|两排|书架|，|上面|也|摆满|了|图书|和|杂志|。|一些|打工|青年|或站|或|蹲|，|认真|地阅|读|，|不时|有|人到|借阅|台前|办理|借书|或|还书|手续|。|南山区|在|深圳|市西边|，|地处|城乡|结合部|，|外来|打|工者|较|多|。|去年|２月|，|南山区|工商|分局|局长|王|安全|发现|分局|对面|的|公园|里|常有|不少|打|工者|业余|时间|闲逛|，|有时|还|滋扰|生事|。|为|了|给|这些|打|工者|提供|一个|充实|自己|的|场|所|，|他|提议|由|全分局|工作|人员|捐款|，|兴建|一个|免费|阅览室|。|领导|带头|，|群众|响应|，|大家|捐款|１．４万|元|，|购买|了|近|千册|图书|。|３月|６日|，|建在|南头|繁华|的|南|新路|和|金鸡路|交叉口|的|阅览室|开放|了|。|从|此|，|这里|每天|都|吸引|了|众多|借书|、|看书|的|人们|，|其中|不仅|有|打|工者|，|还|有|机关|干部|、|公司|职员|和|个|体户|。|到|了|夏天|，|由于|阅览室|所|在|地|被|工程|征用|，|南山区|工商|分局|便|把|阅览室|迁到|了|桂庙|。|阅览室|的|管理|人员|是|两|名|青年|，|男|的|叫|张|攀|，|女|的|叫|赵阳|。|张|攀|自己|就|是|湖北来|的|打|工者|，|听|说|南山区|工商|分局|办|免费|阅览室|，|便|主动|应|聘来|服务|。|阅览室|每天|从|早９时|开到|晚１０时|，|夜里|张|攀|就|住|在|这里|。|他谈|起|阅览室|里|的|图书|，|翻着|一|本本|的|借阅|名册|，|如数|家珍|，|对|图书|和|工作|